In [ ]:
import pandas as pd

In [ ]:
techniques = ["T1003.002-1", "T1053.005-1", "T1569.002-1", "T1082-1", "T1036-1", "T1090.001-3", "T1078.001-2", "T1112-1", "T1562-1", "T1112-4"]

for tech in techniques:
    data = pd.read_csv(f"d:/UPJS_STUDY/MGR/diplomovka/csv_artifacts/artwithsysmon_1000x/{tech}/EventLogs/EventLogs_output_{tech}_1000x.csv")
    data = data.drop([
            "RecordNumber",
            "EventRecordId",
            "Channel",
            "ProcessId",
            "ThreadId",
            "Computer",
            "ChunkNumber",
            "UserId",
            "MapDescription",
            "UserName",
            "RemoteHost",
            "PayloadData1",
            "PayloadData2",
            "PayloadData3",
            "PayloadData4",
            "PayloadData5",
            "PayloadData6",
            "ExecutableInfo",
            "HiddenRecord",
            "SourceFile",
            "ExtraDataOffset"
        ], axis=1
    )

    sorted_data = data.sort_values(by=['TimeCreated'])
    # sorted_data.iloc[[0, -1]]

    # filter only rows between MYSCRIPTSTART and MYSCRIPTEND
    date_start = ''
    date_end = ''

    df = pd.DataFrame(columns=['TimeCreated', 'EventId', 'Level', 'Provider', 'Keywords', 'Payload', 'ID_run'])
    counter = 1

    for index, row in sorted_data.iterrows():   
        if row['Provider'] == 'MYSCRIPTSTART':
            date_start = row['TimeCreated']
            
        if row['Provider'] == 'MYSCRIPTEND':
            date_end = row['TimeCreated']
        
        if date_start and date_end:
            mask = (sorted_data['TimeCreated'] > date_start) & (sorted_data['TimeCreated'] < date_end)
            subset = sorted_data.loc[mask]
            array = [counter] * len(subset) 
            subset.loc[:, 'ID_run'] = list(array)

            df = pd.concat([df, subset])
            
            date_start = ''
            date_end = ''
            counter = counter + 1

    df.to_csv(f"./artifacts_small_1000x/{tech}_small.csv")